### Sustainability Aware Asset Management: **Group A (Minimum-Variance Portfolio)**
#### **Part I - Standard Asset Allocation**

In [78]:
## Packages lists:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.optimize import minimize

## Importing all the dataframes from data.ipynb
%run data.ipynb

In [79]:
## Converting into returns using the simple return definition. Also replacing the NaN in the first period by 0.
df_m = monthly_return.pct_change().fillna(0)
df_m.index = pd.to_datetime(df_m.index)

df_y = yearly_return.pct_change().fillna(0)

### Part I - Standard Asset Allocation

#### 1.1 - Construction of a minimum variance portfolio

As we are computing the minimum variance portfolio out-of-sample, we use the first 6 years of monthly returns (from Jan. 2000 to Dec. 2005) to compute the vector of expected returns and the covariance matrix

In [80]:
## Sampling the dataframe from Jan. 2000 to Dec. 2005
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2005, 12, 31)
stocks = df_m.columns
sample_m = df_m[df_m.index.isin(pd.date_range(start, end))]

## Checking if tau is equal to 72
tau = len(sample_m)
tau == 72

True

We computed the expected returns as:
$$
\hat{\mu}_{Y+1} = \frac{1}{\tau}\sum_{k=0}^{\tau-1} R_{t-k}
$$

In [81]:
## It is simply a basic mean function.
mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T

The covariance matrix is computed as:
$$
\Sigma_{Y+1} = \frac{1}{\tau}\sum_{k=0}^{\tau-1} (R_{t-k} - \hat{\mu}_{Y+1})'(R_{t-k} - \hat{\mu}_{Y+1})
$$

In [83]:
## We use the parameter bias=True, because we divide the sum by 1/tau instead of 1/(tau-1)
excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)
covmat = 1/tau * excess_returns.T @ excess_returns
## We convert it into a dataframe

In [84]:
## We can see that some values are missing in our covariance matrix, so we need to investigate.
print(covmat.isnull().values.any())

## After some investigations, we can see that the following stocks are causing problems.
## Their TRI are very small, thus these stocks are insignificant, and removing them isn't that much of a problem.
stocktodrop = ['ICT.TUNGGAL PRAKARSA', 'ASTRA AGRO LESTARI',
               'BANK PAN INDONESIA', 'BANK NEGARA INDONESIA',
               'GLOBAL MEDIACOM', 'LIPPO KARAWACI',
               'ASTRA INTERNATIONAL', 'SK HYNIX', 'HYUNDAI ENGR.& CON.']

## 

True


In [85]:
## We run again the previous codes, after dropping the problematic stocks.

## Importing the original data, removing the stocks, then calculating the returns
df_m = monthly_return.drop(stocktodrop, axis=1)
df_m.index = pd.to_datetime(df_m.index)
df_m = df_m.pct_change().fillna(0)
stocks = df_m.columns

## Sampling the dataframe from Jan. 2000 to Dec. 2005
sample_m = df_m[df_m.index.isin(pd.date_range(start, end))]

## Calculating the expected returns, then the excess_returns
mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T
excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)

## Calculating the covariance matrix
covmat = 1/tau * excess_returns.T @ excess_returns

## Control if there are any missing values.
print(covmat.isnull().values.any())

False


For the allocation, we use the following maximizazion problem, while restricting the optimal weights to be positive:

$$\min_{{{\alpha_{Y}}}}\quad \sigma^{2}_{p,Y+1} = \alpha'_{Y}\Sigma_{Y+1}\alpha_{Y}$$

$$\text{s.t.}\quad \alpha'_{Y}e = 1$$

$$\text{s.t.}\quad  \alpha_{i,Y} \ge 0 \quad \text{for all i}$$

In [8]:
# Define objective function (portfolio variance)
def portfolio_variance(weights, covmat):
    return np.dot(weights.T, np.dot(covmat, weights))

# Define constraint (sum of weights equals 1)
def constraint(weights):
    return np.sum(weights) - 1
    
# Number of assets
n_assets = len(covmat)

# Initial guess for weights
initial_weights = np.ones(n_assets) / n_assets

# Define bounds for weights (0 to 1) long-only portfolio
bounds = [(0, None)] * n_assets

# Perform optimization
result = minimize(portfolio_variance, initial_weights, args=(covmat,), constraints={'type': 'eq', 'fun': constraint}, bounds=bounds)

# Get optimal weights
optimal_weights = result.x

# Print results
print(optimal_weights)

[0.00000000e+00 5.53090005e-03 2.07991957e-18 ... 0.00000000e+00
 0.00000000e+00 9.45540327e-22]


In [95]:
weights = pd.read_csv('Data/firstweight.csv', index_col=0)
weights

,weight
SCHLUMBERGER,0.000000e+00
ALUAR,5.530900e-03
BBVA BANCO FRANCES,2.079920e-18
TERNIUM ARGENTINA SOCIEDAD ANONIMA,1.891144e-05
FLUGHAFEN WIEN,5.030383e-18
...,...
ALTRON LIMITED A,0.000000e+00
ABSA GROUP,0.000000e+00
NORTHAM PLATINUM HLDGS,0.000000e+00
AVENG,0.000000e+00


In [96]:
weights.sum(axis=0)

weight    1.0
dtype: float64